In [1]:
import random
import numpy as np
import pandas as pd
import open3d as o3d

from pathlib import Path
from itertools import product


random.seed(42)
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Error)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
import utils
import calibration_utils

import importlib
importlib.reload(utils)
importlib.reload(calibration_utils)

<module 'calibration_utils' from '/home/tom/Documents/UNT/csce6260/projects/kitti-experiments/calibration_utils.py'>

In [3]:
base = Path.home() / "kitti"
train_dir = base / "training"
velo_dir = train_dir / "velodyne"
point_cloud_train_files = sorted(velo_dir.glob("*.bin"))
calib_dir = train_dir / "calib"
calib_train_files = sorted(calib_dir.glob("*.txt"))
train_labels_dir = train_dir / "label_2"
kitti_train_labels = sorted(train_labels_dir.glob("*.txt"))

In [4]:
subset_size = 300
assert len(point_cloud_train_files) == len(calib_train_files) == len(kitti_train_labels)
random_indices = random.sample(range(len(point_cloud_train_files)), subset_size)
point_cloud_subset = [point_cloud_train_files[i] for i in random_indices]
calib_subset = [calib_train_files[i] for i in random_indices]
labels_subset = [kitti_train_labels[i] for i in random_indices]
print(f"Subset size: {len(point_cloud_subset)} point clouds, "
      f"{len(calib_subset)} calibs, {len(labels_subset)} labels")

Subset size: 300 point clouds, 300 calibs, 300 labels


In [5]:
def downsample(points, voxel_size=0.2):
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(points[:, :3])
    point_cloud = point_cloud.voxel_down_sample(voxel_size=voxel_size)
    return point_cloud

def segment(point_cloud):
    plane_model, inliers = point_cloud.segment_plane(distance_threshold=0.3, ransac_n=3, num_iterations=150)
    outlier_cloud = point_cloud.select_by_index(inliers, invert=True)
    return outlier_cloud

def cluster(outlier_cloud, eps=0.45, min_points=10):
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Debug) as cm:
        cluster_labels = np.array(outlier_cloud.cluster_dbscan(eps=eps, min_points=min_points, print_progress=False))
    return cluster_labels

In [6]:
MAX_POINTS = 300
MIN_POINTS = 25

def get_bbox_predictions(cluster_labels):
    predicted_bboxes = []
    clusters = pd.Series(range(len(cluster_labels))).groupby(cluster_labels, sort=False).apply(list).tolist()
    for cluster in clusters:
        nb_points = len(outlier_cloud.select_by_index(cluster).points)
        if (nb_points > MIN_POINTS and nb_points < MAX_POINTS):
            sub_cloud = outlier_cloud.select_by_index(cluster)
            pred_bbox = sub_cloud.get_axis_aligned_bounding_box()            
            predicted_bboxes.append(pred_bbox)
    return predicted_bboxes

In [7]:
def get_cam_to_velo_transform(calib):
    velo_to_cam = calib['Tr_velo_to_cam']
    velo_to_cam = np.vstack((velo_to_cam, [0, 0, 0, 1]))
    cam_to_velo = calibration_utils.inverse_rigid_transform(velo_to_cam)    
    return cam_to_velo

def get_ground_truth_bboxes(labels, cam_to_velo):
    gt_bboxes = []
    for label in labels:
        corners_3d_velo = calibration_utils.compute_box_3d(label, cam_to_velo)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(corners_3d_velo)
        gt_bbox = pcd.get_axis_aligned_bounding_box()
        gt_bboxes.append(gt_bbox)
    return gt_bboxes

In [8]:
voxel_sizes = [0.05, 0.1, 0.15, 0.2]
dbscan_eps = [0.35, 0.4, 0.45, 0.5, 0.55]
min_points = [5, 10, 15, 20]

hyperparameter_grid = list(product(voxel_sizes, dbscan_eps, min_points))
print(f"\nTotal hyperparameter combinations: {len(hyperparameter_grid)}")
all_scores = []
for i, (v, eps, mp) in enumerate(hyperparameter_grid):
    total_TP, total_FP, total_FN = 0, 0, 0
    for cloud_file, calib_file, label_file in list(zip(point_cloud_subset, calib_subset, labels_subset)):
        points = utils.read_velodyne_bin(cloud_file)
        point_cloud = downsample(points, voxel_size=v)
        outlier_cloud = segment(point_cloud)
        cluster_labels = cluster(outlier_cloud, eps, mp)
        predicted_bboxes = get_bbox_predictions(cluster_labels)
        predicted_bboxes = list(filter(lambda b: b.min_bound[0] > 0, predicted_bboxes))
        labels = utils.parse_label_file(label_file)
        calib = utils.parse_calib_file(calib_file)
        cam_to_velo = get_cam_to_velo_transform(calib)
        gt_bboxes = get_ground_truth_bboxes(labels, cam_to_velo)
        results = utils.evaluate_metrics(gt_bboxes, predicted_bboxes, iou_threshold=0.2)
        total_TP += results['TP']
        total_FP += results['FP']
        total_FN += results['FN']
    precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) > 0 else 0
    recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) > 0 else 0
    f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    all_scores.append({
        "voxel_size": v,
        "eps": eps,
        "min_points": mp,
        "precision": precision,
        "recall": recall,
        "f1": f1_score,
        "raw_results": {'TP': total_TP, 'FP': total_FP, 'FN': total_FN}
    })
    break


Total hyperparameter combinations: 80
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 515
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 90
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 603
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 208
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 485
[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 446
[Open3D DEBUG] Precompute neighbors.
[Open3D

In [ ]:
all_scores_sorted = sorted(all_scores, key=lambda x: x["f1"], reverse=True)
top_n = 5
print(f"Top {top_n} configurations based on F1 score:")
for rank, entry in enumerate(all_scores_sorted[:top_n], 1):
    print(f"{rank}: voxel_size={entry['voxel_size']}, eps={entry['eps']}, min_points={entry['min_points']}")
    print(f"    Precision={entry['precision']:.3f}, Recall={entry['recall']:.3f}, F1={entry['f1']:.3f}")
    print(f"    Raw results: {entry['raw_results']}")